# GBQ


## Parameters


In [1]:
project_id = 'ioracle'
db_name = 'aapl_data'


## Import data

In [2]:
import yfinance as yf
import pandas as pd
from google_auth_oauthlib import flow

In [3]:
def _get_dataframe(ticker_name, start, end):
    """
    get_dataframe(ticker_name, start, end)
    Downloads OHLC,adj close and volume from yahoo finance
    returns dataframe

    """
    df = yf.download(ticker_name, start=start, end=end)
    return df


def _get_start_end(kwargs):
    '''
    get_start_end(kwargs)
    from kwargs, get start, end dates
    if not stated, will return default values
    return start, end dates
    '''
    start = kwargs.get('start', "2017-01-01")
    end = kwargs.get('end', "2022-01-01") #not inclusive
    return start, end
         

def save_local(ticker_name, path_filename, **kwargs):
    """
    save_local(path_filename, ticker_name, **kwargs)
    save df to local path
    """
    start, end = _get_start_end(kwargs)
      
    df = _get_dataframe(ticker_name, start=start, end=end)
    if len(df) != 0:
        df.to_csv(path_filename)
        print(f"{ticker_name} from {start} to {end} saved to {path_filename}")
        

def _get_credentials(secrets='client_secrets.json'):
    """
    get_credentials(secrets='client_secrets.json')
    open browser to authenticate file transfer,
    saves credentials to global var
    """
    appflow = flow.InstalledAppFlow.from_client_secrets_file(
        secrets, scopes=["https://www.googleapis.com/auth/bigquery"]
    )
    appflow.run_local_server()
    global credentials 
    credentials = appflow.credentials
    

def save_gbq(ticker_name, table_name, **kwargs):
    """
    save_to_gbq(table_name, project_id=None)
    convert df to uploadable format for gbq
    """
    #check if credentials exist, create credentials if necssary
    try: 
        credentials
    except:
        _get_credentials()
    
    start, end = _get_start_end(kwargs)
    project_id = kwargs.get('project_id', "ioracle")
    
            
    temp = _get_dataframe(ticker_name, start=start, end=end)
    
##    For testing, avoid keep downloading data
#     temp = pd.read_csv('play.csv')

    if len(temp) != 0: # check that df is not empty
        temp = temp.rename(columns={'Adj Close': 'Adj_Close'}).reset_index()
        temp.to_gbq(f'{project_id}.main.{table_name}', 
                    project_id=project_id, 
                    credentials=credentials,
                    table_schema = [{'name': 'Date','type':'DATE'}] #hard code schema for date from DATETIME to DATE
                   )
        

def read_local(path_filename):
    """
    read_local(path_filename)
    reads the csv file and parses date col as date, setting the date as the index
    returns the df
    """
    df = pd.read_csv(path_filename)
    df['Date'] = pd.to_datetime(df['Date'])
    return df.set_index('Date')
    

# read from gbq (undo changes)
def read_gbq(table_name, **kwargs):
    #check if credentials exist, create credentials if necssary
    try: 
        credentials
    except:
        _get_credentials()
        
    project_id = kwargs.get('project_id', "ioracle")
    
    sql = f"SELECT * FROM `{project_id}.main.{table_name}` "
    
    df = pd.read_gbq(sql, project_id=project_id, credentials=credentials)
    df = df.sort_values('Date').set_index('Date')
    
    return df
    


In [4]:

df = read_gbq(db_name)
df

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=284152559720-e7dh1rm43tgs62jta8f2kkk4r2r7rq0h.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=d1qk2OYfMep9vDJOMVRg7zbNlDxtvT&access_type=offline


,Open,High,Low,Close,Adj_Close,Volume
Date,,,,,,
2017-01-03,28.950001,29.082500,28.690001,29.037500,27.332474,115127600
2017-01-04,28.962500,29.127501,28.937500,29.004999,27.301880,84472400
2017-01-05,28.980000,29.215000,28.952499,29.152500,27.440720,88774400
2017-01-06,29.195000,29.540001,29.117500,29.477501,27.746635,127007600
2017-01-09,29.487499,29.857500,29.485001,29.747499,28.000776,134247600
...,...,...,...,...,...,...
2021-12-27,177.089996,180.419998,177.070007,180.330002,180.330002,74919600
2021-12-28,180.160004,181.330002,178.529999,179.289993,179.289993,79144300
2021-12-29,179.330002,180.630005,178.139999,179.380005,179.380005,62348900


## Get splits for model validation

In [5]:
#OOP
from datetime import datetime as dt

class train_val_split:
    
    def __init__(self, df, duration=30, window=1, prediction_period=5, start = '2018-01-01', end = '2020-03-31'):
        self.duration = duration # training period, for dates
        self.window = window # rolling window freq, for dates
        self.prediction_period = prediction_period # prediciton horizon, for dates
        self.start = start
        self.end = end
        self.df = df.loc[(df.index >= start) & (df.index <= end)].reset_index()
        self.start_ind = self.df[self.df.Date>=self.start].index.min()
        
    @staticmethod
    def _strfdate(date):
        return dt.strftime(date, '%Y-%m-%d')
           
    
    def split_by_date(self):
        self.end_ind = self.start_ind + self.duration
        
        dates = []
        
        while self.end_ind <=  len(self.df) - self.prediction_period:
            date_start = self._strfdate(self.df.Date[self.start_ind])
            date_end = self._strfdate(self.df.Date[self.end_ind])
            
            dates.append([date_start, date_end])
            
            self.start_ind += self.window
            self.end_ind = self.start_ind + self.duration
            
        return dates
    
    def _chk_split(self, date_split):
        for n in range(len(date_split)-1):
            if date_split[n] >= date_split[n+1]:
                return False
        if date_split[0] < self.start:
            return False
        if date_split[-1] > self.end:
            return False
        return True
            
    
    def split_by_index(self, date_split = ['2018-09-30','2019-06-30','2020-03-31']):
        
        if not self._chk_split(date_split):
            return "Check date split again"
        
        ind_out = []            
        for n, date in enumerate(date_split):
            self.train_end_ind = self.df[self.df.Date<=date].index.max()
            
            if n < len(date_split)-1:
                val_end = self.df[self.df.Date<=date_split[n+1]].index.max()
            else:
                val_end = self.df.index.max()
            
            #check boundary_dates
            print(self.df.Date[self.train_end_ind], self.df.Date[self.train_end_ind+1], self.df.Date[val_end])
                
            ind_out.append((list(range(self.start_ind, self.train_end_ind+1)), list(range(self.train_end_ind+1, val_end+1))))
            
        return ind_out
    
    def get_val_map(self, start='2020-06-01', end='2020-12-31'):
        start_ind = self.df[self.df.Date>=start].index.min()
        end_ind = self.df[self.df.Date<=end].index.max()
        
        return {self._strfdate(self.df.Date[n-5]):self._strfdate(self.df.Date[n]) for n in range(start_ind, end_ind+1)}                
            

In [6]:
splitter  = train_val_split(df)
dates = splitter.split_by_date()
dates

[['2018-01-02', '2018-02-14'],
 ['2018-01-03', '2018-02-15'],
 ['2018-01-04', '2018-02-16'],
 ['2018-01-05', '2018-02-20'],
 ['2018-01-08', '2018-02-21'],
 ['2018-01-09', '2018-02-22'],
 ['2018-01-10', '2018-02-23'],
 ['2018-01-11', '2018-02-26'],
 ['2018-01-12', '2018-02-27'],
 ['2018-01-16', '2018-02-28'],
 ['2018-01-17', '2018-03-01'],
 ['2018-01-18', '2018-03-02'],
 ['2018-01-19', '2018-03-05'],
 ['2018-01-22', '2018-03-06'],
 ['2018-01-23', '2018-03-07'],
 ['2018-01-24', '2018-03-08'],
 ['2018-01-25', '2018-03-09'],
 ['2018-01-26', '2018-03-12'],
 ['2018-01-29', '2018-03-13'],
 ['2018-01-30', '2018-03-14'],
 ['2018-01-31', '2018-03-15'],
 ['2018-02-01', '2018-03-16'],
 ['2018-02-02', '2018-03-19'],
 ['2018-02-05', '2018-03-20'],
 ['2018-02-06', '2018-03-21'],
 ['2018-02-07', '2018-03-22'],
 ['2018-02-08', '2018-03-23'],
 ['2018-02-09', '2018-03-26'],
 ['2018-02-12', '2018-03-27'],
 ['2018-02-13', '2018-03-28'],
 ['2018-02-14', '2018-03-29'],
 ['2018-02-15', '2018-04-02'],
 ['2018-

## Get best Architecture from GBQ

In [7]:
from google.cloud import bigquery as bq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file('service-account-file.json')


client = bq.Client(credentials= credentials,project=project_id)

In [17]:
from tqdm import tqdm
from time import sleep

test_model_name = 'arima_test'

coeff_list = []

for date in tqdm(dates):
    query1 = f"DROP MODEL IF EXISTS `{project_id}.main.{test_model_name}`"
    client.query(query1)

    start_date = date[0]
    end_date = date[1]

    query2=f"""
                CREATE OR REPLACE MODEL `{project_id}.main.{test_model_name}`
                OPTIONS(MODEL_TYPE='ARIMA_PLUS',
                         time_series_timestamp_col='Date',
                         time_series_data_col='Adj_Close',
                         DATA_FREQUENCY = 'DAILY',
                         HOLIDAY_REGION = 'GLOBAL',
                         CLEAN_SPIKES_AND_DIPS = FALSE) AS
                SELECT Date, Adj_Close
                FROM `{project_id}.main.{db_name}`
                WHERE Date Between '{start_date}' AND '{end_date}'
                ORDER BY Date ASC
        """
    client.query(query2)

    query3 = f"SELECT * FROM ML.EVALUATE(MODEL `{project_id}.main.{test_model_name}`)"
    while True:
        try:
            temp_df = client.query(query3).to_dataframe()
            break
        except:
            sleep(1)
        
    coeff = temp_df.sort_values('log_likelihood', ascending = False).iloc[0,:4].values       
    coeff_list.append((tuple(coeff[:3]),coeff[3]))
    

100%|███████████████████████████████████████████████████████████████████████████████| 531/531 [1:18:49<00:00,  8.91s/it]


In [28]:
from statistics import mode

best_params = mode([tuple(x) for x in coeff_list])
NON_SEASONAL_ORDER, INCLUDE_DRIFT = best_params[0], best_params[1] 

## Get Error Score

In [87]:


val_split = train_val_split(df, end='2021-12-31')
val_map = val_split.get_val_map(end='2021-12-31')
val_map

{'2020-05-22': '2020-06-01',
 '2020-05-26': '2020-06-02',
 '2020-05-27': '2020-06-03',
 '2020-05-28': '2020-06-04',
 '2020-05-29': '2020-06-05',
 '2020-06-01': '2020-06-08',
 '2020-06-02': '2020-06-09',
 '2020-06-03': '2020-06-10',
 '2020-06-04': '2020-06-11',
 '2020-06-05': '2020-06-12',
 '2020-06-08': '2020-06-15',
 '2020-06-09': '2020-06-16',
 '2020-06-10': '2020-06-17',
 '2020-06-11': '2020-06-18',
 '2020-06-12': '2020-06-19',
 '2020-06-15': '2020-06-22',
 '2020-06-16': '2020-06-23',
 '2020-06-17': '2020-06-24',
 '2020-06-18': '2020-06-25',
 '2020-06-19': '2020-06-26',
 '2020-06-22': '2020-06-29',
 '2020-06-23': '2020-06-30',
 '2020-06-24': '2020-07-01',
 '2020-06-25': '2020-07-02',
 '2020-06-26': '2020-07-06',
 '2020-06-29': '2020-07-07',
 '2020-06-30': '2020-07-08',
 '2020-07-01': '2020-07-09',
 '2020-07-02': '2020-07-10',
 '2020-07-06': '2020-07-13',
 '2020-07-07': '2020-07-14',
 '2020-07-08': '2020-07-15',
 '2020-07-09': '2020-07-16',
 '2020-07-10': '2020-07-17',
 '2020-07-13':

In [88]:
df['y_actual'] = df['Adj_Close'].shift(-5)
df.tail(10)

,Open,High,Low,Close,Adj_Close,Volume,y_actual
Date,,,,,,,
2021-12-17,169.929993,173.470001,169.690002,171.139999,171.139999,195432700,180.330002
2021-12-20,168.279999,170.580002,167.460007,169.750000,169.750000,107499100,179.289993
2021-12-21,171.559998,173.199997,169.119995,172.990005,172.990005,91185900,179.380005
2021-12-22,173.039993,175.860001,172.149994,175.639999,175.639999,92135300,178.199997
2021-12-23,175.850006,176.850006,175.270004,176.279999,176.279999,68227500,177.570007
2021-12-27,177.089996,180.419998,177.070007,180.330002,180.330002,74919600,NaN
2021-12-28,180.160004,181.330002,178.529999,179.289993,179.289993,79144300,NaN
2021-12-29,179.330002,180.630005,178.139999,179.380005,179.380005,62348900,NaN
2021-12-30,179.470001,180.570007,178.089996,178.199997,178.199997,59773000,NaN


In [89]:
def get_prediction_dict(df, dic, end_date, tgt_date):
    df['tgt_date'] = df['forecast_timestamp'].apply(lambda x: dt.strftime(x, '%Y-%m-%d'))
    dic[end_date] = df.set_index('tgt_date').loc[tgt_date, 'forecast_value']
    

In [91]:
start_date = '2018-01-01'
model_name = 'arima_model'
pred_dict = {}

for k, v in tqdm(val_map.items()):
    
    query4 = f"DROP MODEL IF EXISTS `{project_id}.main.{model_name}`"
    client.query(query4)

    query5 = f"""
            CREATE MODEL IF NOT EXISTS `{project_id}.main.{model_name}`
                OPTIONS(MODEL_TYPE='ARIMA_PLUS',
                         time_series_timestamp_col='Date',
                         time_series_data_col='Adj_Close',
                         DATA_FREQUENCY = 'DAILY',
                         HOLIDAY_REGION = 'GLOBAL',
                         CLEAN_SPIKES_AND_DIPS = FALSE,
                         AUTO_ARIMA = FALSE,
                         NON_SEASONAL_ORDER = {NON_SEASONAL_ORDER},
                         INCLUDE_DRIFT = {INCLUDE_DRIFT}) AS
                SELECT Date, Adj_Close
                FROM `{project_id}.main.{db_name}`
                WHERE Date Between '{start_date}' AND '{k}'
                ORDER BY Date ASC
        """

    client.query(query5).to_dataframe()

    query6 = f"""SELECT * FROM ML.FORECAST(MODEL `{project_id}.main.{model_name}`,
                                        STRUCT(20 AS horizon)
                )"""

    while True:
        try:
            forecast_df = client.query(query6).to_dataframe()
            break
        except:
            sleep(1)
            
    get_prediction_dict(forecast_df, pred_dict, k, v)

100%|███████████████████████████████████████████████████████████████████████████████| 402/402 [1:05:14<00:00,  9.74s/it]


In [92]:
pred_df = pd.DataFrame(pred_dict,index=['arima_predicted']).T
pred_df.index = pd.to_datetime(pred_df.index)
len(pred_df)

402

In [93]:
pred_df = pred_df.join(df[['y_actual']], how='inner')
pred_df

,arima_predicted,y_actual
2020-05-22,76.415184,79.688095
2020-05-26,80.892000,80.057014
2020-05-27,79.470875,80.497726
2020-05-28,78.713294,79.804459
2020-05-29,77.870976,82.077385
...,...,...
2021-12-17,164.961542,180.330002
2021-12-20,164.208711,179.289993
2021-12-21,168.186904,179.380005
2021-12-22,167.876654,178.199997


In [101]:
pred_df.to_csv('aapl_arima_pred.csv')

In [106]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

map1 = val_split.get_val_map()
comp_df = pred_df[(pred_df.index >= min(map1)) & (pred_df.index <= max(map1))]
# print(comp_df)

for error in ['mean_absolute_error', 'mean_squared_error']:
    print(error)
    print(eval(f"{error}(comp_df['y_actual'], comp_df['arima_predicted'])"))
    print()

mean_absolute_error
4.5926510975119665

mean_squared_error
39.80454612009772



In [107]:
map2 = val_split.get_val_map(start="2021-01-01", end="2021-12-31")
comp_df = pred_df[(pred_df.index >= min(map2)) & (pred_df.index <= max(map2))]
print(comp_df)

for error in ['mean_absolute_error', 'mean_squared_error']:
    print(error)
    print(eval(f"{error}(comp_df['y_actual'], comp_df['arima_predicted'])"))
    print()

            arima_predicted    y_actual
2020-12-24       131.526587  128.617111
2020-12-28       139.727464  130.207291
2020-12-29       131.877740  125.824318
2020-12-30       131.590721  130.117859
2020-12-31       129.311885  131.240906
...                     ...         ...
2021-12-17       164.961542  180.330002
2021-12-20       164.208711  179.289993
2021-12-21       168.186904  179.380005
2021-12-22       167.876654  178.199997
2021-12-23       168.054580  177.570007

[252 rows x 2 columns]
mean_absolute_error
4.660548678297973

mean_squared_error
38.96172179468353

